In [2]:
import numpy as np
import torch
import torch.nn as nn

In [3]:
#input must be tensor with float (each bit)
def IndA_diff(input):
    weights = torch.tensor([[-7.8433e-04, 37.006, 37.006, 37.006, 37.005, 37.006, 37.006, 37.007]])
    bias = torch.tensor([-131.3873])
    linear_output = torch.matmul(input, weights.t()) + bias
    output = torch.sigmoid(linear_output)

    return output

In [4]:
#input must be tensor with float (each bit)
def IndB_diff(input):
    weights = torch.tensor([[-0.4066, -0.4066, -0.4066, -0.4066, -0.4066, -0.4066, -0.4066, 20.1961]])
    bias = torch.tensor([-8.4722])
    linear_output = torch.matmul(input, weights.t()) + bias
    output = torch.sigmoid(linear_output)

    return output

In [5]:
def IndC_diff(bits,cnt):
   
    return 1

In [6]:
#input must be tensor with float (each bit)
def IndD_diff(input):
    weights = torch.tensor([[ 0.3999,0.3999,0.3999, 0.3999, 0.3999, 0.3999, 0.3999,-15.5119]])
    bias = torch.tensor([6.1550])
    linear_output = torch.matmul(input, weights.t()) + bias
    output = torch.sigmoid(linear_output)

    return output

In [7]:
def IndE_diff(bits,cnt):
    
    return 1

In [8]:
def NegBit_diff(x):
    return 1 / (1 + np.exp(-15.43*x + 7.51))

In [38]:
def Real2Bit_diff(x):
    return 1 / (1 + np.exp(-(-5.43 + 10.81*x)))

0.49375032550048975

In [10]:
def XNOR_diff(x1, x2):
    fc1_weight = torch.tensor([[4.7662,  4.8464],[-0.1215,  1.3168],[ 2.6322,  2.3068],[-0.0951, -0.4683]])
    fc1_bias = torch.tensor([-1.5654,  0.3904, -3.6841, -0.0062])
    fc2_weight = torch.tensor([[-5.6381,  1.5594,  5.0998, -0.2311]])
    fc2_bias = torch.tensor([1.7433])

    x = torch.tensor([x1, x2]).float()
    fc1_output = torch.sigmoid(torch.matmul(x, fc1_weight.transpose(0, 1)) + fc1_bias)
    fc2_output = torch.sigmoid(torch.matmul(fc1_output, fc2_weight.transpose(0, 1)) + fc2_bias)

    return fc2_output.item()

In [11]:
def XOR_diff(x1, x2):
    fc1_weight = torch.tensor([[-3.8973,  4.0330],[-4.1031,  3.8928],[-0.6775,  0.8108],[ 2.3834, -2.4550]])
    fc1_bias = torch.tensor([ 2.2917, -2.1209, -0.6751, -1.6954])
    fc2_weight = torch.tensor([[-4.5110,  5.7411,  1.0059,  2.4664]])
    fc2_bias = torch.tensor([0.8839])

    x = torch.tensor([x1, x2]).float()
    fc1_output = torch.sigmoid(torch.matmul(x, fc1_weight.transpose(0, 1)) + fc1_bias)
    fc2_output = torch.sigmoid(torch.matmul(fc1_output, fc2_weight.transpose(0, 1)) + fc2_bias)

    return fc2_output.item()

In [12]:
def OR_diff(x1, x2):
    weight = torch.tensor([[13.5223, 13.5223]])
    bias = torch.tensor([-6.3024])
    x = torch.tensor([x1, x2]).float()
    output = torch.sigmoid(torch.matmul(x, weight.transpose(0, 1)) + bias)

    return output.item()

In [13]:
#input must be float (each bit)
def q_m_diff(input):
    t_input = torch.tensor(input)
    output = torch.tensor([t_input[0], 0, 0, 0, 0, 0, 0, 0, 0])
    for bit in range(1, 8):
        output[bit] = IndA_diff(t_input) * Real2Bit_diff(XNOR_diff(output[bit-1], t_input[bit])) + (1 - IndA_diff(t_input)) * Real2Bit_diff(XOR_diff(output[bit-1],t_input[bit]))
    output[8] = 1 - IndA_diff(t_input)
    return output


In [15]:
def TMDS_diff(pixel_bits,cnt):
    bits_inversos = pixel_bits[::-1]
    q_m = q_m_diff(bits_inversos)
    output = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
    for bit in range(0,8):
        output[bit] = q_m[bit] * OR_diff(IndB_diff(q_m[:8]),(1-IndC_diff(q_m[:8],cnt))) + NegBit_diff(q_m[bit]) * OR_diff((1 - IndB_diff(q_m[:8])),IndC_diff(q_m[:8],cnt))
    output[8] = q_m[8]
    output[9] = IndE_diff(q_m[:8],cnt) * NegBit_diff(q_m[8]) + (1 - IndE_diff(q_m[:8],cnt)) * IndC_diff(q_m[:8],cnt)
    new_cnt = IndE_diff(q_m[:8],cnt) * ((cnt + torch.sum(q_m[:8] == 0).item() - torch.sum(q_m[:8] == 1).item()) * IndD_diff(q_m[:8]) + (cnt + torch.sum(q_m[:8] == 1).item() - torch.sum(q_m[:8] == 0).item()) * (1 -IndD_diff(q_m[:8]))) + (1 - IndE_diff(q_m[:8],cnt)) * ((cnt + 2 * q_m[8] + torch.sum(q_m[:8] == 0).item() - torch.sum(q_m[:8] == 1).item()) * IndC_diff(q_m[:8],cnt) + (cnt - 2 * NegBit_diff(q_m[8]) + torch.sum(q_m[:8] == 1).item() - torch.sum(q_m[:8] == 0).item()) * (1 - IndC_diff(q_m[:8],cnt)) )
    return output,new_cnt

pixel_bits = [1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0]
cnt=0
TMDS_diff(pixel_bits,cnt)

([tensor(0.0005),
  tensor(0.0008),
  tensor(0.0008),
  tensor(0.0008),
  tensor(0.0008),
  tensor(0.0008),
  tensor(1.0013),
  tensor(0.0007),
  tensor(1.),
  tensor(0.9996)],
 tensor([0.9965]))

In [ ]:
def g_simplified(input):
   
    return output

In [71]:
def Pixel2Bit_diff(pixel):
    output = [0, 0, 0, 0, 0, 0, 0, 0]
    for i in range(1,9):
        output[i-1] = 1 / (1 + np.exp(-(11*(pixel-2**(8-i)+0.5))))  #0.5 to adjust for the sigmoid so at 127.5 is at 0.5 and at 128 is already at 1
        if pixel >= 2**(8-i):
            pixel = pixel - 2**(8-i)
    return output